In [284]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import sklearn
from scipy import stats
import statsmodels.formula.api as sm

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from sklearn import linear_model
import patsy
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split

%config InlineBackend.figure_format = 'png' #set 'png' here when working on notebook
%matplotlib inline

In [309]:
data = pd.read_excel("NewRawData.xlsx")

In [310]:
data['Exter Qual'] = data['Exter Qual'].replace('Ex', 5)
data['Exter Qual'] = data['Exter Qual'].replace('Gd', 4)
data['Exter Qual'] = data['Exter Qual'].replace('TA', 3)
data['Exter Qual'] = data['Exter Qual'].replace('Fa', 2)
data['Exter Qual'] = data['Exter Qual'].replace('Po', 1)
data['Exter Cond'] = data['Exter Cond'].replace('Ex', 5)
data['Exter Cond'] = data['Exter Cond'].replace('Gd', 4)
data['Exter Cond'] = data['Exter Cond'].replace('TA', 3)
data['Exter Cond'] = data['Exter Cond'].replace('Fa', 2)
data['Exter Cond'] = data['Exter Cond'].replace('Po', 1)
data['Bsmt Qual'] = data['Bsmt Qual'].replace('Ex', 5)
data['Bsmt Qual'] = data['Bsmt Qual'].replace('Gd', 4)
data['Bsmt Qual'] = data['Bsmt Qual'].replace('TA', 3)
data['Bsmt Qual'] = data['Bsmt Qual'].replace('Fa', 2)
data['Bsmt Qual'] = data['Bsmt Qual'].replace('Po', 1)
data['Bsmt Qual'] = data['Bsmt Qual'].replace('No Basement', 0)
data['Bsmt Cond'] = data['Bsmt Cond'].replace('Ex', 5)
data['Bsmt Cond'] = data['Bsmt Cond'].replace('Gd', 4)
data['Bsmt Cond'] = data['Bsmt Cond'].replace('TA', 3)
data['Bsmt Cond'] = data['Bsmt Cond'].replace('Fa', 2)
data['Bsmt Cond'] = data['Bsmt Cond'].replace('Po', 1)
data['Bsmt Cond'] = data['Bsmt Cond'].replace('No Basement', 0)

In [200]:
from pandas import ExcelWriter

writer = ExcelWriter('NewData.xlsx')
data.to_excel(writer,'Sheet1')
writer.save()

In [311]:
singlefeatures = data.columns[:-1]
features = list(singlefeatures)
for i in range(len(features)):
    features[i] = "".join(features[i].split())
    features[i] = features[i].replace('.','DOT')


In [312]:
singlefeatures = features
features.append('SalePrice')
data.columns = features
train, test = train_test_split(data, test_size=0.2)#it is not time-series data
train.to_csv('train.csv')
test.to_csv('test.csv')

In [313]:
singlefeatures = list(set(features)-{'SalePrice'})

In [314]:
X = patsy.dmatrix('+'.join(singlefeatures)+'-1',train)
Y = train['SalePrice']
lasso = linear_model.LassoCV(cv =10, alphas = [500]).fit(X,Y)
lassofeatures = np.array(singlefeatures)[np.abs(lasso.coef_)>1e-10]
print lassofeatures

[u'Neighborhood_NridgHt' u'OverallCond' u'GrLivArea' u'BsmtCond'
 u'OverallQual' u'Neighborhood_Gilbert' u'TotalBsmtSF'
 u'HouseStyle_1Story' u'Neighborhood_StoneBr' u'YrSold' u'BsmtQual'
 u'BldgType_1Fam' u'GarageCars' u'YearBuilt' u'HouseStyle_2Story'
 u'BedroomAbvGr' u'LotArea' u'LotShape_IR1' u'ExterQual'
 u'Neighborhood_OldTown' u'Neighborhood_NoRidge' u'SaleType_New'
 u'Neighborhood_Crawfor']


In [315]:
lm = smf.ols('SalePrice ~ '+' + '.join(lassofeatures), data=train).fit()
print 'Here is in-sample R-square:'+str(lm.rsquared)
rsq = 1. - np.sum((lm.predict(test)-test['SalePrice'])**2)/np.sum((
    test['SalePrice'].mean()-test['SalePrice'])**2)
print 'Here is the out-of-sample R-square:'+str(rsq)

Here is in-sample R-square:0.8402156172809604
Here is the out-of-sample R-square:0.887839645654


In [324]:
data = pd.read_excel("NotRawData.xlsx")
train, test = train_test_split(data, test_size=0.2)#i
singlefeatures = data.columns[:-1]
features = list(singlefeatures)
for i in range(len(features)):
    features[i] = "".join(features[i].split())
    features[i] = features[i].replace('.','DOT')
singlefeatures = list(set(features)-{'SalePrice'})
features.append('SalePrice')
data.columns = features


In [327]:
'SalePrice ~ '+' + '.join(singlefeatures)

u'SalePrice ~ HeatingQC_Gd + 3SsnPorch + LotShape_Reg + SaleType_COD + ExterCond_Gd + HouseStyle_SFoyer + OpenPorchSF + Exterior1st_HdBoard + BsmtFinType2_ALQ + SaleType_ConLw + Condition2_Artery + Condition1_RRAe + PoolQC_Fa + SaleCondition_AdjLand + Neighborhood_GrnHill + BsmtFinType2_LwQ + Condition1_RRAn + Exterior1st_AsbShng + MSSubClass_SPLITFOYER + Neighborhood_Landmrk + Functional_Maj2 + EnclosedPorch + Exterior2nd_Stucco + BsmtFullBath + Exterior2nd_AsbShng + BsmtQual_NoBasement + LotFrontage + Exterior2nd_WdSdng + BsmtFinType1_NoBasement + RoofMatl_Tar&Grv + KitchenAbvGr + Neighborhood_MeadowV + SaleType_ConLI + BsmtExposure_Gd + Neighborhood_NoRidge + ExterQual_TA + HouseStyle_1Story + BldgType_2fmCon + SaleType_ConLD + Exterior2nd_Stone + ScreenPorch + MasVnrType_BrkFace + PoolQC_NoPool + CentralAir_N + Fence_GdWo + FullBath + SaleType_Oth + BsmtFinType1_Unf + GarageCond_Po + Utilities_NoSewr + Condition2_RRAe + YearBuilt + PoolQC_Gd + Exterior2nd_WdShng + KitchenQual_Po + 

In [328]:
lm = smf.ols('SalePrice ~ '+' + '.join(singlefeatures), data=train).fit()
print 'Here is in-sample R-square:'+str(lm.rsquared)
rsq = 1. - np.sum((lm.predict(test)-test['SalePrice'])**2)/np.sum((
    test['SalePrice'].mean()-test['SalePrice'])**2)
print 'Here is the out-of-sample R-square:'+str(rsq)

PatsyError: intercept term cannot interact with anything else
    SalePrice ~ HeatingQC_Gd + 3SsnPorch + LotShape_Reg + SaleType_COD + ExterCond_Gd + HouseStyle_SFoyer + OpenPorchSF + Exterior1st_HdBoard + BsmtFinType2_ALQ + SaleType_ConLw + Condition2_Artery + Condition1_RRAe + PoolQC_Fa + SaleCondition_AdjLand + Neighborhood_GrnHill + BsmtFinType2_LwQ + Condition1_RRAn + Exterior1st_AsbShng + MSSubClass_SPLITFOYER + Neighborhood_Landmrk + Functional_Maj2 + EnclosedPorch + Exterior2nd_Stucco + BsmtFullBath + Exterior2nd_AsbShng + BsmtQual_NoBasement + LotFrontage + Exterior2nd_WdSdng + BsmtFinType1_NoBasement + RoofMatl_Tar&Grv + KitchenAbvGr + Neighborhood_MeadowV + SaleType_ConLI + BsmtExposure_Gd + Neighborhood_NoRidge + ExterQual_TA + HouseStyle_1Story + BldgType_2fmCon + SaleType_ConLD + Exterior2nd_Stone + ScreenPorch + MasVnrType_BrkFace + PoolQC_NoPool + CentralAir_N + Fence_GdWo + FullBath + SaleType_Oth + BsmtFinType1_Unf + GarageCond_Po + Utilities_NoSewr + Condition2_RRAe 